## API Wrapper

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

In [2]:
secrets_file = open("C:\\Users\\franc\\OneDrive\\Área de Trabalho\Data Analytics\\Week 2\\6.05 API Wrappers, Spotipy\\secrets.txt","r")
string = secrets_file.read()
string.split('\n')
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1]

In [3]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['csecret']))

In [4]:
#playlist = sp.user_playlist_tracks("spotify", "1OfeGWEHudCL9mtqoR6Os8") #test 1

In [5]:
#playlist = sp.user_playlist_tracks("spotify", "1C49yxU1XBkoq5yaVDbJwx") #test 2 

In [6]:
playlist = sp.user_playlist_tracks("spotify", "69fEt9DN5r4JQATi52sRtq") #list with a song from each Spotify genre

In [7]:
playlist.keys() 

dict_keys(['href', 'items', 'limit', 'next', 'offset', 'previous', 'total'])

In [8]:
playlist["items"][0].keys()

dict_keys(['added_at', 'added_by', 'is_local', 'primary_color', 'track', 'video_thumbnail'])

In [9]:
playlist["total"] 

5888

In [10]:
len(playlist["items"])

100

In [11]:
playlist['next'] #link to the next set of results

'https://api.spotify.com/v1/playlists/69fEt9DN5r4JQATi52sRtq/tracks?offset=100&limit=100&additional_types=track'

In [12]:
from random import randint
from time import sleep

def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,3))
    return tracks

In [13]:
all_tracks = get_playlist_tracks("69fEt9DN5r4JQATi52sRtq")
len(all_tracks)

5888

In [14]:
all_tracks[0]["track"]['album']['artists'][0]['uri']

'spotify:artist:3L7fQ04gd9c8Yc5GSrzqzZ'

An idea for collecting as many songs as possible is to start with all the songs of a big, diverse playlist and then go to every artist present in the playlist and grab every song of every album of that artist. The amount of songs you'll be collecting per playlist will grow exponentially!

Playlist already too big. Processing is already too long.

In [15]:
# get track info -  artist, song name and uri:
def get_track_info(tracks):
    
    song_uri = []
    artists = []
    artist_uri = []
    song_name = []
    popularity = []
    artist_uri = []
    
    for i in range(len(all_tracks)):
        #get rid of duplicate songs
        if tracks[i]["track"]["uri"] not in song_uri:
            song_name.append(tracks[i]["track"]["name"])
            song_uri.append(tracks[i]["track"]["uri"])
            artists.append(tracks[i]["track"]["artists"][0]["name"])
            artist_uri.append(tracks[i]["track"]['album']['artists'][0]['uri'])
            popularity.append(tracks[i]["track"]["popularity"])
            
    track_info = {'artist' : artists , 'artist_uri': artist_uri, 'song' : song_name, 'song_uri':song_uri, 'popularity' : popularity}
    
    return track_info

In [16]:
def get_features(song_uri):
    features = []
    for i in song_uri:
        features.append(sp.audio_features(i))
    return features

In [17]:
def get_genres(artist_uri):
    genres = []
    for i in artist_uri:
        genres.append(sp.artist(i))
    return genres    

In [25]:
track_info = get_track_info(all_tracks)

In [19]:
len(track_info["song"])

5887

In [20]:
artist_uri = track_info['artist_uri']

In [21]:
genres = get_genres(artist_uri)

In [22]:
len(genres)

5887

In [26]:
#Genres dataframe

In [29]:
track_info['genres'] = genres

In [ ]:
track_info

In [32]:
df_track_info = pd.DataFrame(track_info)

In [33]:
df_track_info

,artist,artist_uri,song,song_uri,popularity,genres
0,Matthias Pintscher,spotify:artist:3L7fQ04gd9c8Yc5GSrzqzZ,Nemeton for solo Percussion,spotify:track:7wmm8ycV3DwStuQK0LGkdH,9,{'external_urls': {'spotify': 'https://open.sp...
1,Marc Van Mourik,spotify:artist:7s9gWdUK3IXsvkQP58Es4H,Pure,spotify:track:4cFb48uJoAaR0ZogBcUTM5,34,{'external_urls': {'spotify': 'https://open.sp...
2,NGT48,spotify:artist:4wZjP2p0Sm5EHM32uuAgGf,青春時計,spotify:track:62bPGqh8IYBpVOzxIxkrXA,31,{'external_urls': {'spotify': 'https://open.sp...
3,Carly Cosgrove,spotify:artist:4h3DxZa1JxZhSB1a64ziKx,Buttersock,spotify:track:5d3g3AivF2agJ99fFRrqkR,34,{'external_urls': {'spotify': 'https://open.sp...
4,Super Power Club,spotify:artist:5vGR79cM4EqZxXgvcExy7f,Seven Nation Army,spotify:track:3MTLncsF4tzTICc05bM034,21,{'external_urls': {'spotify': 'https://open.sp...
...,...,...,...,...,...,...
5882,Edith Lefel,spotify:artist:0nuevJYykJ4x09p8UIqmcS,Si seulement,spotify:track:200n4pv4B6L2LfcCaBcybk,31,{'external_urls': {'spotify': 'https://open.sp...
5883,Paille,spotify:artist:22odTnbJhyDIQFnp8z1PgA,"Coucou freestyle, pt. 1",spotify:track:13uQwct5dTKpYAHVn5AysX,26,{'external_urls': {'spotify': 'https://open.sp...
5884,Sissy Fox,spotify:artist:2PiLMskYjBRTHmY6PoR31x,Young When The Rain Falls,spotify:track:4T4V2y8HwBOyjFUt4REcuk,2,{'external_urls': {'spotify': 'https://open.sp...
5885,Three AM Fuxk,spotify:artist:46WXN5rRkUX5bCvwsjics4,X,spotify:track:6vuEjpJFDnQfgSv2HYyC3h,52,{'external_urls': {'spotify': 'https://open.sp...


In [35]:
df_genres = pd.DataFrame(dict(df_track_info['genres'])).T

In [36]:
#df_genres['genres'] = df_genres['genres'].str.split(',')
#df_genres['genres'] = df_genres.drop('genres',1).join(df_genres.genres.str.join('|').str.get_dummies())
#df_genres.isna().sum()
df_genre = pd.get_dummies(df_genres['genres'].apply(pd.Series), prefix='', prefix_sep='').sum(level=0, axis=1)

C:\Users\franc\AppData\Local\Temp/ipykernel_6844/139770807.py:4: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df_genre = pd.get_dummies(df_genres['genres'].apply(pd.Series), prefix='', prefix_sep='').sum(level=0, axis=1)


In [37]:
#Features dataframe

In [41]:
song_uri = track_info['song_uri']

In [42]:
features = get_features(song_uri)
len(features)

5887

In [43]:
def flatten(input_list):
    return [item for sublist in input_list for item in sublist]

In [44]:
features = flatten(features)

In [45]:
df_track_info['features'] = features

In [46]:
df_features = pd.DataFrame(dict(df_track_info['features'])).T

In [47]:
df_features

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.39,0.0692,0,-23.312,1,0.0905,0.435,0.504,0.0822,0.0433,76.465,audio_features,7wmm8ycV3DwStuQK0LGkdH,spotify:track:7wmm8ycV3DwStuQK0LGkdH,https://api.spotify.com/v1/tracks/7wmm8ycV3DwS...,https://api.spotify.com/v1/audio-analysis/7wmm...,740413,4
1,0.283,0.00966,0,-28.107,1,0.0327,0.993,0.807,0.0884,0.108,103.468,audio_features,4cFb48uJoAaR0ZogBcUTM5,spotify:track:4cFb48uJoAaR0ZogBcUTM5,https://api.spotify.com/v1/tracks/4cFb48uJoAaR...,https://api.spotify.com/v1/audio-analysis/4cFb...,259042,5
2,0.694,0.761,5,-4.087,0,0.0372,0.446,0,0.289,0.55,114.947,audio_features,62bPGqh8IYBpVOzxIxkrXA,spotify:track:62bPGqh8IYBpVOzxIxkrXA,https://api.spotify.com/v1/tracks/62bPGqh8IYBp...,https://api.spotify.com/v1/audio-analysis/62bP...,208467,4
3,0.487,0.75,0,-8.348,1,0.0516,0.0582,0.000008,0.283,0.823,127.769,audio_features,5d3g3AivF2agJ99fFRrqkR,spotify:track:5d3g3AivF2agJ99fFRrqkR,https://api.spotify.com/v1/tracks/5d3g3AivF2ag...,https://api.spotify.com/v1/audio-analysis/5d3g...,138688,4
4,0.646,0.279,4,-3.575,1,0.0545,0.149,0.915,0.0993,0.715,120.063,audio_features,3MTLncsF4tzTICc05bM034,spotify:track:3MTLncsF4tzTICc05bM034,https://api.spotify.com/v1/tracks/3MTLncsF4tzT...,https://api.spotify.com/v1/audio-analysis/3MTL...,147587,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5882,0.571,0.75,7,-5.987,0,0.0307,0.419,0,0.104,0.563,87.982,audio_features,200n4pv4B6L2LfcCaBcybk,spotify:track:200n4pv4B6L2LfcCaBcybk,https://api.spotify.com/v1/tracks/200n4pv4B6L2...,https://api.spotify.com/v1/audio-analysis/200n...,242867,4
5883,0.602,0.795,2,-4.828,1,0.111,0.0458,0,0.166,0.858,127.993,audio_features,13uQwct5dTKpYAHVn5AysX,spotify:track:13uQwct5dTKpYAHVn5AysX,https://api.spotify.com/v1/tracks/13uQwct5dTKp...,https://api.spotify.com/v1/audio-analysis/13uQ...,149381,5
5884,0.866,0.695,8,-7.347,1,0.0357,0.335,0.884,0.126,0.899,117.031,audio_features,4T4V2y8HwBOyjFUt4REcuk,spotify:track:4T4V2y8HwBOyjFUt4REcuk,https://api.spotify.com/v1/tracks/4T4V2y8HwBOy...,https://api.spotify.com/v1/audio-analysis/4T4V...,211160,4
5885,0.523,0.391,4,-11.769,0,0.03,0.437,0.492,0.205,0.0378,109.952,audio_features,6vuEjpJFDnQfgSv2HYyC3h,spotify:track:6vuEjpJFDnQfgSv2HYyC3h,https://api.spotify.com/v1/tracks/6vuEjpJFDnQf...,https://api.spotify.com/v1/audio-analysis/6vuE...,169038,4


In [48]:
df_song_features = pd.concat([df_track_info, df_features], axis = 1 )
df_song_features_genres = pd.concat([df_track_info, df_features, df_genre], axis = 1 )
df_song_features = df_song_features.drop(['features', 'song_uri'], axis = 1)
df_song_features_genres = df_song_features.drop(['features', 'song_uri', 'genres'], axis = 1)

In [49]:
df_song_features.isna().sum()

artist                0
artist_uri            0
song                  0
popularity            0
danceability          0
                     ..
uk tech house         0
olympia wa indie      0
psychedelic punk      0
nordic ambient        0
polish modern jazz    0
Length: 5582, dtype: int64

In [50]:
df_song_features_genres.to_csv("df_song_features_genres.csv")

In [ ]:
df_song_features.to_csv("df_song_features.csv")